In [1]:
# Dependencies
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
path = 'Resources'
fastfood_county = pd.DataFrame()

# get all csv files
csv_files = glob.glob(os.path.join(path, "*.csv"))


# Loop through files to store all csvs in a df
for f in csv_files:
    # read the file into dataframe
    file_load = pd.read_csv(f)
    
    # add column County and populate with filename - .csv
    # filename = f.strip("Resources/")
    # filename = filename.strip(".csv")
    # file_load["County"] = filename
    
    #transpose the data and remove the first column
    file_load = file_load.T
    file_load = file_load.iloc[1: , :]
    
    # add it to the main dataframe, ignoring the first 
    fastfood_county = fastfood_county.append(file_load, ignore_index=True)
        
    # print success
    # print(f"loaded {f}")

In [3]:
fastfood_county.rename(columns={ fastfood_county.columns[0]: "County" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[1]: "Fast_food_2011" }, inplace = True)
fastfood_county.rename(columns={ fastfood_county.columns[2]: "Fast_food_2016" }, inplace = True)

fastfood_county = fastfood_county[["County","Fast_food_2011","Fast_food_2016"]]

In [4]:
fastfood_county.head()

,County,Fast_food_2011,Fast_food_2016
0,Adams,31,34
1,Alexander,0,1
2,Bond,7,9
3,Boone,26,27
4,Brown,2,4


In [5]:
fastfood_count_2011_df=fastfood_county[["County","Fast_food_2011"]]
fastfood_count_2011_df["Year"] ="2011"
fastfood_count_2011_df = fastfood_count_2011_df.rename( columns = {"Fast_food_2011":"Restaurant Count"})

fastfood_count_2011_df.head()

,County,Restaurant Count,Year
0,Adams,31,2011
1,Alexander,0,2011
2,Bond,7,2011
3,Boone,26,2011
4,Brown,2,2011


In [6]:
fastfood_count_2016_df=fastfood_county[["County","Fast_food_2016"]]
fastfood_count_2016_df["Year"] ="2016"
fastfood_count_2016_df = fastfood_count_2016_df.rename( columns = {"Fast_food_2016":"Restaurant Count"})

fastfood_count_2016_df.head()

,County,Restaurant Count,Year
0,Adams,34,2016
1,Alexander,1,2016
2,Bond,9,2016
3,Boone,27,2016
4,Brown,4,2016


In [7]:
fast_food_11_16_df = pd.concat([fastfood_count_2011_df, fastfood_count_2016_df], ignore_index=True)
fast_food_11_16_df

,County,Restaurant Count,Year
0,Adams,31,2011
1,Alexander,0,2011
2,Bond,7,2011
3,Boone,26,2011
4,Brown,2,2011
...,...,...,...
199,Whiteside,33,2016
200,Will,501,2016
201,Williamson,48,2016
202,Winnebago,213,2016


In [8]:
#now bring in FIPS number to prepare to get count by county
IL_Zipcode_county = "IL_Zipcode_countyName.csv"
IL_Zipcode_county_df = pd.read_csv(IL_Zipcode_county)
IL_Zipcode_county_df.head()

,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity
0,60001,17111,McHenry,IL,60001 Alden,42.421141,-88.616252,60033.0,0,0,0
1,60002,17097,Lake,IL,60002 Antioch,42.474263,-88.103323,NaN,0,0,0
2,60004,17031,Cook,IL,60004 Arlington Heights,42.111424,-87.980372,NaN,0,0,0
3,60005,17031,Cook,IL,60005 Arlington Heights,42.064974,-87.982208,NaN,0,0,0
4,60006,17031,Cook,IL,60006 Arlington Heights,42.088133,-87.993611,60005.0,0,0,0


In [9]:
# discovered some inconsistencies in the county name La Salle in future steps....making a change here for consistency
IL_Zipcode_county_df["county2"] = np.where(IL_Zipcode_county_df["CountyName"] == "La Salle", "LaSalle", IL_Zipcode_county_df["CountyName"])
IL_Zipcode_county_df.head()

,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity,county2
0,60001,17111,McHenry,IL,60001 Alden,42.421141,-88.616252,60033.0,0,0,0,McHenry
1,60002,17097,Lake,IL,60002 Antioch,42.474263,-88.103323,NaN,0,0,0,Lake
2,60004,17031,Cook,IL,60004 Arlington Heights,42.111424,-87.980372,NaN,0,0,0,Cook
3,60005,17031,Cook,IL,60005 Arlington Heights,42.064974,-87.982208,NaN,0,0,0,Cook
4,60006,17031,Cook,IL,60006 Arlington Heights,42.088133,-87.993611,60005.0,0,0,0,Cook


In [10]:
df_grouped = IL_Zipcode_county_df.groupby("county2")
FIPS_grouped_df=df_grouped.max()

In [11]:
merge_11_16_FIPS_df = pd.merge(fast_food_11_16_df, FIPS_grouped_df, how="left", left_on="County", right_on="county2")
merge_11_16_FIPS_df

,County,Restaurant Count,Year,ZIP,FIPS,CountyName,St,Name,Lat,Long,Enc_ZIP,Pops,Area,PopDensity
0,Adams,31,2011,62376,17001,Adams,IL,62376 Ursa,40.163633,-90.975425,62301.0,0,0,0
1,Alexander,0,2011,62993,17003,Alexander,IL,62993 Unity,37.334611,-89.181001,62990.0,0,0,0
2,Bond,7,2011,62284,17005,Bond,IL,62284 Smithboro,39.027642,-89.269052,NaN,0,0,0
3,Boone,26,2011,61065,17007,Boone,IL,61065 Poplar Grove,42.399403,-88.739276,NaN,0,0,0
4,Brown,2,2011,62378,17009,Brown,IL,62378 Versailles,40.008266,-90.656159,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Whiteside,33,2016,61283,17195,Whiteside,IL,61283 Tampico,41.862918,-89.689498,61081.0,0,0,0
200,Will,501,2016,60586,17197,Will,IL,60586 Plainfield,41.715613,-87.603510,60436.0,0,0,0
201,Williamson,48,2016,62974,17199,Williamson,IL,62974 Pittsburg,37.829687,-88.802417,62948.0,0,0,0
202,Winnebago,213,2016,61132,17201,Winnebago,IL,61132 Loves Park,42.488968,-88.963462,61111.0,0,0,0


In [12]:
# Export the dataframe to csv
merge_11_16_FIPS_df.to_csv("merge_11_16_FIPS_df.csv")

In [13]:
fast_food_11_16_FIPS_df = merge_11_16_FIPS_df.loc[:,["Restaurant Count", "FIPS","Year"]]
fast_food_11_16_FIPS_df

,Restaurant Count,FIPS,Year
0,31,17001,2011
1,0,17003,2011
2,7,17005,2011
3,26,17007,2011
4,2,17009,2011
...,...,...,...
199,33,17195,2016
200,501,17197,2016
201,48,17199,2016
202,213,17201,2016


In [14]:
# Now bring in the 2021 fast food data, which we had to pull from the yelp API
# Need to clean this data to filter on illinois locations only and eliminate duplicate restaurant id's

Fast_food_21_data = "yelp_fastfood_with_duplicates.csv"

In [15]:
Fast_food_21_df = pd.read_csv(Fast_food_21_data)
Fast_food_21_df.head()

,Unnamed: 0,id,name,Address,zip code,state,lat,lng
0,0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.414234,-88.613502
1,1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033.0,IL,42.409128,-88.612985
2,2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.406180,-88.614069
3,3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.408491,-88.612911
4,4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.405404,-88.613068


In [16]:
# clean the data to filter on state = IL, and eliminate the unnamed column
Fast_food_21_IL_df = Fast_food_21_df.loc[   Fast_food_21_df["state"] == "IL", ["id","name","Address","zip code","state","lat","lng"] ]  
Fast_food_21_IL_df.head()

,id,name,Address,zip code,state,lat,lng
0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.414234,-88.613502
1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033.0,IL,42.409128,-88.612985
2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.406180,-88.614069
3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.408491,-88.612911
4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033.0,IL,42.405404,-88.613068


In [17]:
# Change the zip code to a string
Fast_food_21_IL_df.loc[:, "zip code"] = Fast_food_21_IL_df["zip code"].astype("str")
Fast_food_21_IL_df["zip code str"] = Fast_food_21_IL_df['zip code'].str[:5]
Fast_food_21_IL_df=Fast_food_21_IL_df.loc[ :, ["id","name","Address","zip code str","state","lat","lng"] ]
Fast_food_21_IL_df = Fast_food_21_IL_df.rename( columns = {"zip code str":"zip code"})
Fast_food_21_IL_df.head()

,id,name,Address,zip code,state,lat,lng
0,O20eGNVmF9FVQd_3t_lcmg,Taco Bell,"['325 S Division St', 'Harvard, IL 60033']",60033,IL,42.414234,-88.613502
1,h0anlZoscW80pfXOjU-pug,McDonald's,"['352 S Division', 'Harvard, IL 60033']",60033,IL,42.409128,-88.612985
2,veyT9UbcbsgPupFhdDUfEw,Dairy Queen,"['605 S Division St', 'Harvard, IL 60033']",60033,IL,42.406180,-88.614069
3,coQSOoH3npgjUPYvHCZuUA,Subway,"['360 S Division St', 'Harvard, IL 60033']",60033,IL,42.408491,-88.612911
4,Jtyjr-eht4t5OVyZZj5WRA,Culver's,"['1060 S Division St', 'Harvard, IL 60033']",60033,IL,42.405404,-88.613068


In [18]:
# check for the number of unique id's using using .nunique()
Fast_food_21_IL_df["id"].nunique()

7424

In [19]:
# need to eliminate duplicate restaurant id's.  Will group by id, and make a dataframe by using the max id

distinct_ids = Fast_food_21_IL_df.groupby("id")
distinct_ids_df = distinct_ids.max()
distinct_ids_df.head()

,name,Address,zip code,state,lat,lng
id,,,,,,
--0OK34lFbyyf4q16yDoGw,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,IL,42.111725,-88.033573
--GvS1gnXSqboHluetk9_Q,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,IL,40.947068,-90.352368
--dIVNPnqmx9l0UipYX6oQ,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,IL,40.157408,-89.374600
--fVs_GNeavkOOybb5WNaA,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,IL,41.781745,-88.183961
--kVRdC7JJ2Ph5UaXQw6aA,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,IL,42.046957,-88.126650


In [20]:
# sort the data and look for strange zip codes
distinct_ids_df.sort_values("zip code")

,name,Address,zip code,state,lat,lng
id,,,,,,
UiQKEUhRjavKEgfe-xBx4Q,Popeyes Louisiana Kitchen,"['980 Indianapolis Blvd', 'Hammond, IL 46320']",46320,IL,41.695881,-87.518176
Y-ApgB3F9x1d2AkC0RKfig,KFC,"['325 W Il Route 173', 'Antioch, IL 60002']",60002,IL,42.471319,-88.094823
nAz9CVQtD_lHfPrqkSyfpw,McDonald's,"['1150 Main St', 'Antioch, IL 60002']",60002,IL,42.471242,-88.094090
X6TKn8wfaUHo-mt8qHuWAA,Burger King,"['338 W Il Rte 173', 'Antioch, IL 60002']",60002,IL,42.470515,-88.094953
MFBl85WYatZoNNvSg4olbQ,Popeyes Louisiana Kitchen,"['483 E Route 173', 'Antioch, IL 60002']",60002,IL,42.466861,-88.064337
...,...,...,...,...,...,...
wkUc3tzylCQEAyONt-reKw,Ned's Shed,"['101 N 1st St', 'Vienna, IL 62995']",62995,IL,37.414634,-88.893445
RR4wdhO8BXalpFPPE2YLKw,McDonald's,"['707 E Vine St', 'Vienna, IL 62995']",62995,IL,37.414934,-88.874011
7Naxbnb4YBoJHSE5c2NlZQ,Subway,"['104 Redbud Ln', 'Vienna, IL 62995']",62995,IL,37.417305,-88.874104


In [21]:
# eliminate the 46320 zip code (atypical for Illinois) and the nan zip code

distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "46320") , :]
distinct_ids_df = distinct_ids_df.loc[(distinct_ids_df["zip code"] != "nan") , :]
distinct_ids_df

,name,Address,zip code,state,lat,lng
id,,,,,,
--0OK34lFbyyf4q16yDoGw,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,IL,42.111725,-88.033573
--GvS1gnXSqboHluetk9_Q,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,IL,40.947068,-90.352368
--dIVNPnqmx9l0UipYX6oQ,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,IL,40.157408,-89.374600
--fVs_GNeavkOOybb5WNaA,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,IL,41.781745,-88.183961
--kVRdC7JJ2Ph5UaXQw6aA,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,IL,42.046957,-88.126650
...,...,...,...,...,...,...
zwDpnlgp2FUUBdNJlxy1eQ,Burger King,"['2701 N Western Ave', 'Chicago, IL 60647']",60647,IL,41.930619,-87.687632
zwlByG9L7fqFFXr_uyXaXg,Bijou Pub,"['225 S Main St', 'Monmouth, IL 61462']",61462,IL,40.910584,-90.648115
zx2Ms-G5pqXI1_A0JVv6gw,Cilantro Taco Grill - Stone Park,"['1500 N Mannheim Rd', 'Stone Park, IL 60165']",60165,IL,41.897586,-87.884143


In [22]:
# convert ZIP and FIPS to strng for easier merging subsequently
IL_Zipcode_county_df.loc[:, "ZIP"] = IL_Zipcode_county_df["ZIP"].astype("str")
IL_Zipcode_county_df.loc[:, "FIPS"] = IL_Zipcode_county_df["FIPS"].astype("str")
IL_Zipcode_county_df.dtypes

ZIP            object
FIPS           object
CountyName     object
St             object
Name           object
Lat           float64
Long          float64
Enc_ZIP       float64
Pops            int64
Area            int64
PopDensity      int64
county2        object
dtype: object

In [23]:
merge_distinct_ids_FIPs_df = pd.merge(distinct_ids_df,IL_Zipcode_county_df,how="left",left_on="zip code", right_on="ZIP" )
merge_distinct_ids_FIPs_df = merge_distinct_ids_FIPs_df[ ["name","Address","zip code","FIPS","CountyName"]]
merge_distinct_ids_FIPs_df

,name,Address,zip code,FIPS,CountyName
0,Popeyes Louisiana Kitchen,"['45 N Northwest Hwy', 'Palatine, IL 60067']",60067,17031,Cook
1,McDonald's,"['1158 E Main St', 'Galesburg, IL 61401']",61401,17095,Knox
2,Hardee's,"['539 Woodlawn Rd', 'Lincoln, IL 62656']",62656,17107,Logan
3,Fire It Up,"['1523 N Aurora Rd', 'Ste 107', 'Naperville, I...",60563,17043,DuPage
4,Jo Jo's Restaurant,"['1935 W Golf Rd', 'Schaumburg, IL 60194']",60194,17031,Cook
...,...,...,...,...,...
7417,Burger King,"['2701 N Western Ave', 'Chicago, IL 60647']",60647,17031,Cook
7418,Bijou Pub,"['225 S Main St', 'Monmouth, IL 61462']",61462,17187,Warren
7419,Cilantro Taco Grill - Stone Park,"['1500 N Mannheim Rd', 'Stone Park, IL 60165']",60165,17031,Cook
7420,Honey-Jam Cafe,"['120 E Boughton Rd', 'Bolingbrook, IL 60440']",60440,17197,Will


In [24]:
# now need to get a count of restaurants by FIPS

distinct_FIPS_2021 = merge_distinct_ids_FIPs_df.groupby("FIPS")
distinct_count_by_FIPS_2021_df = distinct_FIPS_2021.count()
distinct_count_by_FIPS_2021_df

,name,Address,zip code,CountyName
FIPS,,,,
17001,36,36,36,36
17005,8,8,8,8
17007,21,21,21,21
17009,3,3,3,3
17011,12,12,12,12
...,...,...,...,...
17195,26,26,26,26
17197,366,366,366,366
17199,38,38,38,38


In [25]:
# Want FIPS in a column
distinct_count_by_FIPS_2021_df["FIPS"] = distinct_count_by_FIPS_2021_df.index
distinct_count_by_FIPS_2021_df

,name,Address,zip code,CountyName,FIPS
FIPS,,,,,
17001,36,36,36,36,17001
17005,8,8,8,8,17005
17007,21,21,21,21,17007
17009,3,3,3,3,17009
17011,12,12,12,12,17011
...,...,...,...,...,...
17195,26,26,26,26,17195
17197,366,366,366,366,17197
17199,38,38,38,38,17199


In [26]:
#do more data cleaning
distinct_count_by_FIPS_2021_df = distinct_count_by_FIPS_2021_df[["name","FIPS"]]
fast_food_21_df=distinct_count_by_FIPS_2021_df.rename(columns={"name": "Restaurant Count"})
fast_food_21_df

,Restaurant Count,FIPS
FIPS,,
17001,36,17001
17005,8,17005
17007,21,17007
17009,3,17009
17011,12,17011
...,...,...
17195,26,17195
17197,366,17197
17199,38,17199


In [27]:
# set up the data frame to look like 2011-16 dataframe
fast_food_21_df.reset_index(drop=True)
fast_food_21_df["Year"] ="2021"
fast_food_21_df=fast_food_21_df.reset_index(drop=True)
fast_food_21_df

,Restaurant Count,FIPS,Year
0,36,17001,2021
1,8,17005,2021
2,21,17007,2021
3,3,17009,2021
4,12,17011,2021
...,...,...,...
89,26,17195,2021
90,366,17197,2021
91,38,17199,2021
92,195,17201,2021


In [28]:
# Now combine with 2011 and 2016....again adding the ignore_index = True to reset the index
fast_food_11_16_21_df = pd.concat([fast_food_11_16_FIPS_df, fast_food_21_df], ignore_index=True)
fast_food_11_16_21_df

,Restaurant Count,FIPS,Year
0,31,17001,2011
1,0,17003,2011
2,7,17005,2011
3,26,17007,2011
4,2,17009,2011
...,...,...,...
293,26,17195,2021
294,366,17197,2021
295,38,17199,2021
296,195,17201,2021


In [29]:
# Change the FIPS to a string
fast_food_11_16_21_df.loc[:, "FIPS"] = fast_food_11_16_21_df["FIPS"].astype("str")
fast_food_11_16_21_df["FIPS str"] = fast_food_11_16_21_df['FIPS'].str[:5]
fast_food_11_16_21_df = fast_food_11_16_21_df.loc[ :, ["Restaurant Count","FIPS str","Year"] ]
fast_food_11_16_21_df = fast_food_11_16_21_df.rename( columns = {"FIPS str":"FIPS"})
fast_food_11_16_21_df

,Restaurant Count,FIPS,Year
0,31,17001,2011
1,0,17003,2011
2,7,17005,2011
3,26,17007,2011
4,2,17009,2011
...,...,...,...
293,26,17195,2021
294,366,17197,2021
295,38,17199,2021
296,195,17201,2021


In [30]:
# Export the dataframe to csv
fast_food_11_16_21_df.to_csv("fast_food_11_16_21.csv")

In [31]:

# NOW ORGANIZE THE DIABETES DATA


In [32]:
# Save path to data set in a variable
Diab2011_data = "2011Diab.csv"
Diab2016_data = "2016Diab.csv"
Diab2021_data = "2021Diab.csv"

In [33]:
# Use Pandas to read 2011 data
Diab2011_df = pd.read_csv(Diab2011_data)
Diab2011_df.head()

,FIPS,State,County,Population,< 18,65 and over,African American,American Indian/ Alaskan Native,Asian,Native Hawaiian/ Other Pacific Islander,...,Population.4,Illiteracy,95% CI - Low.4,95% CI - High.4,Population.5,Liquor Stores,Liquor Store Rate,No. Drive Alone,Workers,% Drive Alone
0,17000,Illinois,NaN,12910409,24.6,12.4,14.5,0.2,4.4,0.0,...,9507861,12.9,10.4,16.6,12842954,1289,10,4374646,5952465,74
1,17001,Illinois,Adams,67054,22.7,17.7,3.5,0.2,0.5,0.0,...,51098,7.9,3.7,14.4,66895,1,1,26847,32620,82
2,17003,Illinois,Alexander,7914,23.5,18.5,33.3,0.3,0.5,0.0,...,6789,15.3,7.4,26.7,8112,2,25,2491,2976,84
3,17005,Illinois,Bond,18103,20.3,14.7,7.3,0.5,0.3,0.0,...,12637,8.6,4.0,15.8,18185,1,5,6882,8222,84
4,17007,Illinois,Boone,54020,29.2,11.4,2.6,0.4,1.4,0.0,...,34823,10.8,5.2,19.4,53892,5,9,18880,22822,83


In [34]:
# Use Pandas to read 2016 data
Diab2016_df = pd.read_csv(Diab2016_data)
Diab2016_df.head()

,FIPS,State,County,# Deaths,Age-Adjusted Mortality,95% CI - Low,95% CI - High,# Deaths.1,Child Mortality Rate,95% CI - Low.1,...,# Non-Hispanic White,% Non-Hispanic White,# Not Proficient in English,% Not Proficient in English,95% CI - Low.12,95% CI - High.12,% Female,# Rural,% Rural,Unnamed: 84
0,17000.0,Illinois,NaN,130047.0,321.0,319.0,323.0,6516.0,53.0,52.0,...,8022946.0,62.3,566446.0,4.7,4.6,4.8,50.9,1477079.0,11.5,NaN
1,17001.0,Illinois,Adams,779.0,337.0,313.0,362.0,24.0,39.0,25.0,...,61897.0,92.4,229.0,0.4,0.1,0.6,51.1,21875.0,32.6,NaN
2,17003.0,Illinois,Alexander,135.0,463.0,382.0,544.0,NaN,NaN,NaN,...,4406.0,58.8,9.0,0.1,0.0,1.1,49.7,5096.0,61.9,NaN
3,17005.0,Illinois,Bond,206.0,345.0,297.0,393.0,NaN,NaN,NaN,...,15165.0,87.8,66.0,0.4,0.0,0.9,47.3,10792.0,60.7,NaN
4,17007.0,Illinois,Boone,486.0,280.0,255.0,305.0,24.0,40.0,26.0,...,40096.0,74.4,2175.0,4.3,3.3,5.3,50.2,10488.0,19.4,NaN


In [35]:
# Use Pandas to read 2021 data
Diab2021_df = pd.read_csv(Diab2021_data)
Diab2021_df.head()

,FIPS,State,County,Life Expectancy,95% CI - Low,95% CI - High,Life Expectancy (AIAN),Life Expectancy (AIAN) 95% CI - Low,Life Expectancy (AIAN) 95% CI - High,Life Expectancy (Asian),...,# Non-Hispanic White,% Non-Hispanic White,# Not Proficient in English,% Not Proficient in English,95% CI - Low.20,95% CI - High.20,% Female,# Rural,% Rural,Unnamed: 276
0,17000,Illinois,NaN,79.4,79.3,79.4,115.1,101.6,128.7,89.6,...,7702651,60.8,495487,4,4.0,4.0,50.9,1477079,11.5,NaN
1,17001,Illinois,Adams,78.3,77.6,78.9,NaN,NaN,NaN,NaN,...,59810,91.4,101,0,0.0,0.0,50.7,21875,32.6,NaN
2,17003,Illinois,Alexander,73.5,71.1,75.9,NaN,NaN,NaN,NaN,...,3672,63.7,6,0,NaN,NaN,51.4,5096,61.9,NaN
3,17005,Illinois,Bond,78.6,77.1,80.0,NaN,NaN,NaN,NaN,...,14354,87.4,23,0,0.0,1.0,48.1,10792,60.7,NaN
4,17007,Illinois,Boone,79.7,78.9,80.5,NaN,NaN,NaN,NaN,...,38474,71.9,1747,3,3.0,4.0,50.0,10488,19.4,NaN


In [36]:
# Filter the 2011 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2011_filtered_df = Diab2011_df[  ["FIPS","State","County","% Diabetic","Population" ]  ]
Diab2011_filtered_df.head()

,FIPS,State,County,% Diabetic,Population
0,17000,Illinois,NaN,7,12910409
1,17001,Illinois,Adams,9,67054
2,17003,Illinois,Alexander,11,7914
3,17005,Illinois,Bond,9,18103
4,17007,Illinois,Boone,7,54020


In [37]:
# Filter the 2016 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2016_filtered_df = Diab2016_df[  ["FIPS","State","County","% Diabetic","Population" ]  ]
Diab2016_filtered_df.head()

,FIPS,State,County,% Diabetic,Population
0,17000.0,Illinois,NaN,10.0,12880580.0
1,17001.0,Illinois,Adams,10.0,66988.0
2,17003.0,Illinois,Alexander,16.0,7492.0
3,17005.0,Illinois,Bond,10.0,17269.0
4,17007.0,Illinois,Boone,10.0,53869.0


In [38]:
# Filter the 2021 data to FIPS, State, County, % Diabetes, and Population Columns
Diab2021_filtered_df = Diab2021_df[  ["FIPS","State","County","% Adults with Diabetes","Population" ]  ]

# rename the % Adults with Diabetes column for consistency with 2011 and 2016
Diab2021_filtered_df = Diab2021_filtered_df.rename( columns = {"% Adults with Diabetes":"% Diabetic"} )
Diab2021_filtered_df.head()

,FIPS,State,County,% Diabetic,Population
0,17000,Illinois,NaN,10,12671821
1,17001,Illinois,Adams,13,65435
2,17003,Illinois,Alexander,5,5761
3,17005,Illinois,Bond,7,16426
4,17007,Illinois,Boone,9,53544


In [39]:
#Add a year column to each dataframe


Diab2011_filtered_df.loc[:,['Year']] = 2011
Diab2016_filtered_df.loc[:,['Year']] = 2016
Diab2021_filtered_df.loc[:,['Year']] = 2021

C:\Users\mpk99\Anaconda3\envs\PythonData38\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [40]:
# Verify year column added correctly for 2011
Diab2011_filtered_df.head()

,FIPS,State,County,% Diabetic,Population,Year
0,17000,Illinois,NaN,7,12910409,2011
1,17001,Illinois,Adams,9,67054,2011
2,17003,Illinois,Alexander,11,7914,2011
3,17005,Illinois,Bond,9,18103,2011
4,17007,Illinois,Boone,7,54020,2011


In [41]:
# Verify year column added correctly for 2016
Diab2016_filtered_df.head()

,FIPS,State,County,% Diabetic,Population,Year
0,17000.0,Illinois,NaN,10.0,12880580.0,2016
1,17001.0,Illinois,Adams,10.0,66988.0,2016
2,17003.0,Illinois,Alexander,16.0,7492.0,2016
3,17005.0,Illinois,Bond,10.0,17269.0,2016
4,17007.0,Illinois,Boone,10.0,53869.0,2016


In [42]:
# Verify year column added correctly for 2021.  I like Rouge One
Diab2021_filtered_df.head()

,FIPS,State,County,% Diabetic,Population,Year
0,17000,Illinois,NaN,10,12671821,2021
1,17001,Illinois,Adams,13,65435,2021
2,17003,Illinois,Alexander,5,5761,2021
3,17005,Illinois,Bond,7,16426,2021
4,17007,Illinois,Boone,9,53544,2021


In [43]:
# now combine the three dataframes into one by doing a union via concatenation
# https://datatofish.com/union-pandas-dataframes/

# start with combining 2011 and 2016

Diab2011_2016_df = pd.concat([Diab2011_filtered_df, Diab2016_filtered_df])
Diab2011_2016_df

,FIPS,State,County,% Diabetic,Population,Year
0,17000.0,Illinois,NaN,7.0,12910409.0,2011
1,17001.0,Illinois,Adams,9.0,67054.0,2011
2,17003.0,Illinois,Alexander,11.0,7914.0,2011
3,17005.0,Illinois,Bond,9.0,18103.0,2011
4,17007.0,Illinois,Boone,7.0,54020.0,2011
...,...,...,...,...,...,...
100,17199.0,Illinois,Williamson,10.0,67008.0,2016
101,17201.0,Illinois,Winnebago,11.0,288542.0,2016
102,17203.0,Illinois,Woodford,9.0,39187.0,2016
103,NaN,NaN,NaN,NaN,NaN,2016


In [44]:
# add 2021
Diab2011_2016_2021_df = pd.concat([Diab2011_2016_df, Diab2021_filtered_df])
Diab2011_2016_2021_df

,FIPS,State,County,% Diabetic,Population,Year
0,17000.0,Illinois,NaN,7.0,12910409.0,2011
1,17001.0,Illinois,Adams,9.0,67054.0,2011
2,17003.0,Illinois,Alexander,11.0,7914.0,2011
3,17005.0,Illinois,Bond,9.0,18103.0,2011
4,17007.0,Illinois,Boone,7.0,54020.0,2011
...,...,...,...,...,...,...
98,17195.0,Illinois,Whiteside,15.0,55175.0,2021
99,17197.0,Illinois,Will,10.0,690743.0,2021
100,17199.0,Illinois,Williamson,13.0,66597.0,2021
101,17201.0,Illinois,Winnebago,11.0,282572.0,2021


In [45]:
# Clean up Diab2011_2016_2021_df to remove na
Diab2011_2016_2021_df = Diab2011_2016_2021_df.dropna(how='any')
Diab2011_2016_2021_df

,FIPS,State,County,% Diabetic,Population,Year
1,17001.0,Illinois,Adams,9.0,67054.0,2011
2,17003.0,Illinois,Alexander,11.0,7914.0,2011
3,17005.0,Illinois,Bond,9.0,18103.0,2011
4,17007.0,Illinois,Boone,7.0,54020.0,2011
5,17009.0,Illinois,Brown,8.0,6591.0,2011
...,...,...,...,...,...,...
98,17195.0,Illinois,Whiteside,15.0,55175.0,2021
99,17197.0,Illinois,Will,10.0,690743.0,2021
100,17199.0,Illinois,Williamson,13.0,66597.0,2021
101,17201.0,Illinois,Winnebago,11.0,282572.0,2021


In [46]:
# Now add a Diabetic Population column (% Diabetic * Population) as it may be useful for analysis

Diab2011_2016_2021_df["Diabetic Population"] = round(1/100 * \
    Diab2011_2016_2021_df["% Diabetic"] * Diab2011_2016_2021_df["Population"],0)

<ipython-input-46-38efaabb9607>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Diab2011_2016_2021_df["Diabetic Population"] = round(1/100 * \


In [47]:
Diab2011_2016_2021_df.dtypes

FIPS                   float64
State                   object
County                  object
% Diabetic             float64
Population             float64
Year                     int64
Diabetic Population    float64
dtype: object

In [48]:
fast_food_11_16_21_df.dtypes

Restaurant Count    object
FIPS                object
Year                object
dtype: object

In [49]:
# make a merge key in both data frames to prep for merging...merging on both FIPS number and Year
# making FIPS a string in order to make the string key

Diab2011_2016_2021_df["FIPS str"] = Diab2011_2016_2021_df["FIPS"].astype("str")
Diab2011_2016_2021_df["merge_key"] = Diab2011_2016_2021_df["FIPS str"].str[:5] + Diab2011_2016_2021_df["Year"].astype("str")
Diab2011_2016_2021_df

<ipython-input-49-8c5400536089>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Diab2011_2016_2021_df["FIPS str"] = Diab2011_2016_2021_df["FIPS"].astype("str")
<ipython-input-49-8c5400536089>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Diab2011_2016_2021_df["merge_key"] = Diab2011_2016_2021_df["FIPS str"].str[:5] + Diab2011_2016_2021_df["Year"].astype("str")


,FIPS,State,County,% Diabetic,Population,Year,Diabetic Population,FIPS str,merge_key
1,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,17001.0,170012011
2,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,17003.0,170032011
3,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,17005.0,170052011
4,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,17007.0,170072011
5,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,17009.0,170092011
...,...,...,...,...,...,...,...,...,...
98,17195.0,Illinois,Whiteside,15.0,55175.0,2021,8276.0,17195.0,171952021
99,17197.0,Illinois,Will,10.0,690743.0,2021,69074.0,17197.0,171972021
100,17199.0,Illinois,Williamson,13.0,66597.0,2021,8658.0,17199.0,171992021
101,17201.0,Illinois,Winnebago,11.0,282572.0,2021,31083.0,17201.0,172012021


In [50]:
# making merge key in the fast food dataframe

fast_food_11_16_21_df["FIPS str"] = fast_food_11_16_21_df["FIPS"].astype("str")
fast_food_11_16_21_df["merge_key"] = fast_food_11_16_21_df["FIPS str"].str[:5] + fast_food_11_16_21_df["Year"].astype("str")
fast_food_11_16_21_df

,Restaurant Count,FIPS,Year,FIPS str,merge_key
0,31,17001,2011,17001,170012011
1,0,17003,2011,17003,170032011
2,7,17005,2011,17005,170052011
3,26,17007,2011,17007,170072011
4,2,17009,2011,17009,170092011
...,...,...,...,...,...
293,26,17195,2021,17195,171952021
294,366,17197,2021,17197,171972021
295,38,17199,2021,17199,171992021
296,195,17201,2021,17201,172012021


In [51]:
# Now merge in the fast food restaurant count data into the diabetes dataframe

merge_diab_fastfood_df = pd.merge(Diab2011_2016_2021_df, fast_food_11_16_21_df, how="left", on="merge_key")
merge_diab_fastfood_df

,FIPS_x,State,County,% Diabetic,Population,Year_x,Diabetic Population,FIPS str_x,merge_key,Restaurant Count,FIPS_y,Year_y,FIPS str_y
0,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,17001.0,170012011,31,17001,2011,17001
1,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,17003.0,170032011,0,17003,2011,17003
2,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,17005.0,170052011,7,17005,2011,17005
3,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,17007.0,170072011,26,17007,2011,17007
4,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,17009.0,170092011,2,17009,2011,17009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,17195.0,Illinois,Whiteside,15.0,55175.0,2021,8276.0,17195.0,171952021,26,17195,2021,17195
302,17197.0,Illinois,Will,10.0,690743.0,2021,69074.0,17197.0,171972021,366,17197,2021,17197
303,17199.0,Illinois,Williamson,13.0,66597.0,2021,8658.0,17199.0,171992021,38,17199,2021,17199
304,17201.0,Illinois,Winnebago,11.0,282572.0,2021,31083.0,17201.0,172012021,195,17201,2021,17201


In [52]:
# select the columns we want

diab_fastfood_df = merge_diab_fastfood_df.loc[ :, [ "FIPS_x", "State", "County", "% Diabetic", "Population", "Year_x", "Diabetic Population","Restaurant Count"]]
diab_fastfood_df

,FIPS_x,State,County,% Diabetic,Population,Year_x,Diabetic Population,Restaurant Count
0,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,31
1,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,0
2,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,7
3,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,26
4,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,2
...,...,...,...,...,...,...,...,...
301,17195.0,Illinois,Whiteside,15.0,55175.0,2021,8276.0,26
302,17197.0,Illinois,Will,10.0,690743.0,2021,69074.0,366
303,17199.0,Illinois,Williamson,13.0,66597.0,2021,8658.0,38
304,17201.0,Illinois,Winnebago,11.0,282572.0,2021,31083.0,195


In [53]:
# clean up the column names

diab_fastfood_rename_df = diab_fastfood_df.rename(columns={"FIPS_x":"FIPS","Year_x":"Year"})
diab_fastfood_rename_df

,FIPS,State,County,% Diabetic,Population,Year,Diabetic Population,Restaurant Count
0,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,31
1,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,0
2,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,7
3,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,26
4,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,2
...,...,...,...,...,...,...,...,...
301,17195.0,Illinois,Whiteside,15.0,55175.0,2021,8276.0,26
302,17197.0,Illinois,Will,10.0,690743.0,2021,69074.0,366
303,17199.0,Illinois,Williamson,13.0,66597.0,2021,8658.0,38
304,17201.0,Illinois,Winnebago,11.0,282572.0,2021,31083.0,195


In [54]:
# some 2021 fast food counts come in blank, as counties with 0 restaurants did not get brought into the 2021 yelp data.
# this is creating nan restaurant counts for some counties in 2021.  Therefore, filling those with 0's

diab_fastfood_rename_df["Restaurant Count"] = diab_fastfood_rename_df["Restaurant Count"].fillna(0)
diab_fastfood_rename_df.head()

,FIPS,State,County,% Diabetic,Population,Year,Diabetic Population,Restaurant Count
0,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,31
1,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,0
2,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,7
3,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,26
4,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,2


In [55]:
# now removing any rows that have blanks (for example FIPS 17000 which is the entire state and not a county)

# THIS IS THE DATAFRAME TO PLOT AND ANALYZE WITH


diab_fastfood_clean_df = diab_fastfood_rename_df.dropna(how='any') 
diab_fastfood_clean_df.head()

,FIPS,State,County,% Diabetic,Population,Year,Diabetic Population,Restaurant Count
0,17001.0,Illinois,Adams,9.0,67054.0,2011,6035.0,31
1,17003.0,Illinois,Alexander,11.0,7914.0,2011,871.0,0
2,17005.0,Illinois,Bond,9.0,18103.0,2011,1629.0,7
3,17007.0,Illinois,Boone,7.0,54020.0,2011,3781.0,26
4,17009.0,Illinois,Brown,8.0,6591.0,2011,527.0,2


In [56]:
# Export the dataframe to csv
diab_fastfood_clean_df.to_csv("diab_fastfood_clean_df.csv")